In [ ]:
class ShearShearDataVector(DataVector):
    def __init__(self, *sources, *args, **kwargs):
        additive_bias = kwargs['additive_bias']
        atmospheric_power_model = kwargs['atmospheric_power_model']
        # not defined here probably: intrinsic_alignment = kwargs['intrinsic alignment']
        
        #self?
        systematics_model_list = []
        systematics_model_list.append(additive_bias)
        systematics_model_list.append(atmospheric_power_model)

        self.source_gal1 = sources[0]
        self.source_gal2 = sources[1]
        
        
        def C_ell_GG(self):
            ## where is cosmology specified??
            n_of_z_1 = self.source_gal1.param_dict['n_of_z']
            n_of_z_2 = self.source_gal2.param_dict['n_of_z']
            return n_of_z_1 + n_of_z_2 #some CCL call
        
        def C_ell_GI(self):
            n_of_z_1 = self.source_gal1.param_dict['n_of_z']
            n_of_z_2 = self.source_gal2.param_dict['n_of_z']
            A_1_1 = self.source_gal1.param_dict['A1']
            A_1_2 = self.source_gal2.param_dict['A1']
            return A_1_1*n_of_z_1 + n_of_z_2 #some CCL call
        def C_ell_IG(self):
            n_of_z_1 = self.source_gal1.param_dict['n_of_z']
            n_of_z_2 = self.source_gal2.param_dict['n_of_z']
            A_1_1 = self.source_gal1.param_dict['A1']
            A_1_2 = self.source_gal2.param_dict['A1']
            return n_of_z_1 + A_1_2*n_of_z_2 #some CCL call
        def C_ell_II(self):
            n_of_z_1 = self.source_gal1.param_dict['n_of_z']
            n_of_z_2 = self.source_gal2.param_dict['n_of_z']
            A_1_1 = self.source_gal1.param_dict['A1']
            A_1_2 = self.source_gal2.param_dict['A1']
            return A_1_1*n_of_z_1 + A_1_2*n_of_z_2 #some CCL call
        
        
        for source in sources:
            try:
                for systematics_model in source.systematics_models:
                    systematics_model_list.append(systematics_model.model_name)
                    
            except AttributeError:
                pass

        DataVector.__init__(self, systematics_models=systematics_model_list)

In [ ]:
class source_galaxy(object):
    def __init__(self, *args, **kwargs):
        self.param_dict = {}
        self.param_dict['intrinsic_model'] = IA_model
        self.param_dict['A1','A2'] = A1, A2
        self.param_dict['multiplicative_bias_model'] = multiplicative_bias_model
        self.param_dict['multiplicative_bias'] = multiplicative_bias
        self.param_dict['n_of_z'] = n_of_z
        self.sample_name = bin_name
        
        self.systematics_model_list = []
        self.systematics_model_list.append(IA_model)
        self.systematics_model_list.append(multiplicative_bias_model)
        
    def intrinsic_alignments(self, *args):
        return self.param_dict['intrinsic_model','A1', 'A2']
        
    def multiplicative_bias(self, *args):
        return self.param_dict['multiplicative_bias']